In [91]:
import cv2
import os
import csv
import numpy as np
import mediapipe as mp
from matplotlib import pyplot as plt
import time

In [92]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [93]:
def mediapipe_detection(frame, model):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    return image, results

In [94]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
   # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [126]:
def record_landmarks(results, action):
    try:
        '''
        pose = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

        right_hand = results.right_hand_landmarks.landmark
        right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand]).flatten())

        left_hand = results.left_hand_landmarks.landmark
        left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in left_hand]).flatten())
        '''

        if results.pose_landmarks:
            pose_row = list(np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten())
        else:
            pose_row = list(np.zeros(132).flatten())
        
        if results.left_hand_landmarks:
            lh_row = list(np.array([[res.x, res.y, res.z, res.visibility] for res in results.left_hand_landmarks.landmark]).flatten())
        else:
            lh_row = list(np.zeros(84).flatten())
        
        if results.right_hand_landmarks:
            rh_row = list(np.array([[res.x, res.y, res.z, res.visibility] for res in results.right_hand_landmarks.landmark]).flatten())
        else:
            rh_row = list(np.zeros(84).flatten())
        
        row = pose_row + right_hand_row + left_hand_row
        row.insert(0, action)

        with open('coords6.csv', mode='a', newline='') as f:
            csv_writer_obj = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer_obj.writerow(row)
            
    except:
        pass





        

In [124]:
len(results.pose_landmarks.landmark)*4

132

# Testing Camera and recording

In [13]:
#TESTING CAMERA

cam = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.5) as holistic:
    while cam.isOpened():
        ret, frame = cam.read()
    
        image, results = mediapipe_detection(frame, holistic)
        
        draw_landmarks(image, results)
        
        cv2.imshow("Sign Language Converter", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        #print(results.right_hand_landmarks) 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cam.release()
cv2.destroyAllWindows()

In [26]:
cam.release()
cv2.destroyAllWindows()

In [127]:
#CREATE HEADER IN CSV
NUM_COORDS = 75
#num_coords = len(results.pose_landmarks.landmark) + len(results.right_hand_landmarks.landmark) + len(results.left_hand_landmarks.landmark)
header_list = ["action"]
for i in range(1, NUM_COORDS+1):
    header_list += ["x{}".format(i),"y{}".format(i),"z{}".format(i),"v{}".format(i)]

with open('coords6.csv', mode='w', newline='') as f:
    csv_writer_obj = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer_obj.writerow(header_list)

In [137]:
DATA_PATH = os.path.join('MP_data')
actions = np.array(["Ambulance"])
no_sequences = 5
sequence_length = 30

In [138]:
#recording per sequence

cam = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.5) as holistic:
    
    for action in actions:
        for sequence in range(no_sequences):
            for frame_no in range(sequence_length):
                
                ret, frame = cam.read()
                image, results = mediapipe_detection(frame, holistic)
                draw_landmarks(image, results)
                
                #print(results.right_hand_landmarks)
                #extract_keypoints(results)
                if frame_no == 0:
                    cv2.putText(image, "Collecting Video for {} Video number {}".format(action, sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                     cv2.putText(image, "Collecting Video for {} Video number {}".format(action, sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)

                record_landmarks(results, action)
                #npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_no))
                #np.save(npy_path, keypoints)
                
                cv2.imshow("Input Sign Language", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    cam.release()
                    cv2.destroyAllWindows()

    cam.release()
    cv2.destroyAllWindows()

In [60]:
#RECORDING WITHOUT ANY SEQUENCE

cam = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.5) as holistic:
    while cam.isOpened():
        ret, frame = cam.read()
    
        image, results = mediapipe_detection(frame, holistic)
        
        draw_landmarks(image, results)

        record_landmarks(results)
        
        cv2.imshow("Sign Language Converter", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        #print(results.right_hand_landmarks) 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cam.release()
cv2.destroyAllWindows()

# SPLIT DATA

In [139]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [140]:
df = pd.read_csv("coords6.csv")

In [141]:
X = df.drop('action', axis=1) #feature
y = df['action'] #target 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [142]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import accuracy_score
import pickle


In [143]:
#TRAINING

pipelines = {
    'lr' : make_pipeline(StandardScaler(), LogisticRegression()),
    'rc' : make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf' : make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb' : make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [144]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [145]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [146]:
fit_models['rf'].predict(X_test)

array(['Ambulance', 'Ambulance', 'Hospital', 'Ambulance', 'Hospital',
       'Ambulance', 'Ambulance', 'Ambulance', 'Ambulance', 'Hospital',
       'Ambulance', 'Hospital', 'Hospital', 'Hospital', 'Ambulance',
       'Hospital', 'Ambulance', 'Hospital', 'Hospital', 'Hospital',
       'Hospital', 'Hospital', 'Hospital', 'Hospital', 'Hospital',
       'Ambulance', 'Ambulance', 'Hospital', 'Ambulance', 'Ambulance',
       'Hospital', 'Hospital', 'Ambulance', 'Ambulance', 'Hospital',
       'Hospital', 'Ambulance', 'Ambulance', 'Ambulance', 'Hospital',
       'Hospital', 'Hospital', 'Ambulance', 'Hospital', 'Hospital',
       'Ambulance', 'Hospital', 'Ambulance', 'Ambulance', 'Ambulance',
       'Ambulance', 'Hospital', 'Hospital', 'Ambulance', 'Ambulance',
       'Hospital', 'Ambulance', 'Hospital', 'Ambulance', 'Hospital',
       'Hospital', 'Ambulance', 'Hospital', 'Ambulance', 'Ambulance',
       'Hospital', 'Ambulance', 'Hospital', 'Ambulance', 'Ambulance',
       'Hospital', 'Hospita

In [147]:
y_test

164    Ambulance
200    Ambulance
73      Hospital
163    Ambulance
115     Hospital
         ...    
83      Hospital
99      Hospital
182    Ambulance
124     Hospital
58      Hospital
Name: action, Length: 90, dtype: object

In [148]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['lr'], f)

In [149]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [150]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

# Real Time Testing

In [151]:
cam = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.5) as holistic:
    while cam.isOpened():
        ret, frame = cam.read()
    
        image, results = mediapipe_detection(frame, holistic)
        draw_landmarks(image, results)
        
        #print(results.right_hand_landmarks)

        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
    
            right_hand = results.right_hand_landmarks.landmark
            right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand]).flatten())
    
            left_hand = results.left_hand_landmarks.landmark
            left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in left_hand]).flatten())
     
            row = pose_row + right_hand_row + left_hand_row
            
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)

            '''   coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)'''
            
            # Get status box
            cv2.rectangle(image, (0,0), (640, 35), (0,0,0, 0.5), -1)

            cv2.putText(image, 'Action:'
                        , (10,25), cv2.FONT_HERSHEY_DUPLEX, 0.85, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (110,25), cv2.FONT_HERSHEY_DUPLEX, 0.85, (255, 255, 255), 1, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'Accuracy:'
                        , (420,25), cv2.FONT_HERSHEY_DUPLEX, 0.85, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (560,25), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 255), 1, cv2.LINE_AA)
        except:
            pass


        cv2.imshow("Sign Language Converter", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cam.release()
cv2.destroyAllWindows()


C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


Hospital [0.11969117 0.88030883]


C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


Hospital [0.36949789 0.63050211]


C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


...

Ambulance [0.86312696 0.13687304]
Ambulance [0.87114004 0.12885996]


C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


Ambulance [0.82229979 0.17770021]
Ambulance [0.93420399 0.06579601]


C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


Ambulance [0.96021966 0.03978034]
